In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess data
data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])



11490434/11490434 [==============================] - 0s 0us/step


In [2]:
# Custom callback to track unchanged weights
class WeightChangeTracker(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Save initial weights
        self.initial_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]

    def on_epoch_end(self, epoch, logs=None):
        unchanged_weights = []
        current_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        print('\n')

        for i, (initial_layer_weights, current_layer_weights) in enumerate(zip(self.initial_weights, current_weights)):
            unchanged = 0
            for w_init, w_curr in zip(initial_layer_weights, current_layer_weights):
                unchanged += np.sum(w_init == w_curr)  # Count unchanged weights

            unchanged_weights.append(unchanged)
            print(f'Epoch {epoch + 1}, Layer {i + 1}: {unchanged} unchanged weights')

        # Update the initial weights for the next epoch
        self.initial_weights = current_weights

# Fit the model with the custom callback
model.fit(training_images, training_labels, epochs=10, callbacks=[WeightChangeTracker()])

# Evaluate the model
model.evaluate(test_images, test_labels)


Epoch 1/10
1858/1875 [============================>.] - ETA: 0s - loss: 0.2630 - accuracy: 0.9244

Epoch 1, Layer 1: 10849 unchanged weights
Epoch 1, Layer 2: 0 unchanged weights
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2622 - accuracy: 0.9245
Epoch 2/10
1854/1875 [============================>.] - ETA: 0s - loss: 0.1153 - accuracy: 0.9659

Epoch 2, Layer 1: 12326 unchanged weights
Epoch 2, Layer 2: 10 unchanged weights
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1148 - accuracy: 0.9661
Epoch 3/10
1859/1875 [============================>.] - ETA: 0s - loss: 0.0808 - accuracy: 0.9758

Epoch 3, Layer 1: 12647 unchanged weights
Epoch 3, Layer 2: 10 unchanged weights
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0806 - accuracy: 0.9758
Epoch 4/10
1860/1875 [============================>.] - ETA: 0s - loss: 0.0601 - accuracy: 0.9818

Epoch 4, Layer 1: 12769 unchanged weights
Epoch 4, Layer 2: 10 unchanged weights
1875/187

[0.070916086435318, 0.9810000061988831]